# Verify Option 8: Full Taxation of Social Security Benefits

This notebook verifies that Option 8 results in `taxable_social_security == social_security` at the national aggregate level.

## 1. Setup and Imports

In [1]:
from policyengine_us import Microsimulation
from src.reforms import get_option8_reform
import numpy as np
import pandas as pd

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Create Reform and Load Simulation

In [2]:
# Create Option 8 reform
reform = get_option8_reform()

# Create microsimulation
print("Loading microsimulation data...")
sim = Microsimulation(reform=reform)
print("Done!")

Loading microsimulation data...
Done!


## 3. Check Reform Parameters

Let's verify that Option 8 sets all the correct parameters to achieve 100% taxation.

In [3]:
print("Reform parameter values:")
print("="*80)
params = reform.parameter_values
ss_params = {k: v for k, v in params.items() if "social_security" in str(k).lower()}

for key, value in sorted(ss_params.items()):
    print(f"  {key}")
    print(f"    {value}")

Reform parameter values:
  gov.irs.social_security.taxability.combined_income_ss_fraction
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.benefit_cap
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.bracket
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.additional.excess
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.base.benefit_cap
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.rate.base.excess
    {'2026-01-01.2100-12-31': 1.0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.HEAD_OF_HOUSEHOLD
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.JOINT
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.SEPARATE
    {'2026-01-01.2100-12-31': 0}
  gov.irs.social_security.taxability.threshold.adjusted_base.main.SINGL

## 4. Calculate Aggregate Values for 2026

In [4]:
year = 2026

print(f"Calculating aggregate values for {year}...")
# Calculate at household level from the start
social_security = sim.calculate("social_security", year, map_to="household")
taxable_social_security = sim.calculate("taxable_social_security", year, map_to="household")

# Aggregate totals
total_ss = social_security.sum()
total_taxable_ss = taxable_social_security.sum()
difference = total_ss - total_taxable_ss
percentage_taxable = (total_taxable_ss / total_ss * 100) if total_ss > 0 else 0

print(f"\nAggregate Results for {year}:")
print(f"  Total Social Security Benefits:     ${total_ss:>20,.0f}")
print(f"  Total Taxable Social Security:      ${total_taxable_ss:>20,.0f}")
print(f"  Difference:                         ${difference:>20,.0f}")
print(f"  Percentage Taxable:                 {percentage_taxable:>20.2f}%")

Calculating aggregate values for 2026...

Aggregate Results for 2026:
  Total Social Security Benefits:     $   1,494,708,375,943
  Total Taxable Social Security:      $   1,493,096,104,367
  Difference:                         $       1,612,271,577
  Percentage Taxable:                                99.89%


## 5. Identify Problem Households

In [5]:
# Find households with SS benefits
has_ss = social_security > 0

# Calculate the ratio of taxable to total SS
ratio = np.divide(
    taxable_social_security,
    social_security,
    out=np.zeros_like(taxable_social_security, dtype=float),
    where=social_security > 0
)

print(f"\nStatistics for households with SS benefits:")
print(f"  Total households with SS: {has_ss.sum():,}")
print(f"  Total SS benefits:        ${social_security[has_ss].sum():,.0f}")
print(f"  Total taxable SS:         ${taxable_social_security[has_ss].sum():,.0f}")
print(f"  Overall percentage:       {(taxable_social_security[has_ss].sum() / social_security[has_ss].sum() * 100):.4f}%")


Statistics for households with SS benefits:
  Total households with SS: 51,746,271.82575705
  Total SS benefits:        $1,494,708,375,943
  Total taxable SS:         $1,493,096,104,367
  Overall percentage:       99.8921%


## 6. Deep Dive: Investigate the Problem Households

Let's examine various characteristics to understand why these households aren't fully taxed.

In [6]:
# Create indices for problem households and comparison group (fully taxed)
# Convert to numpy arrays for indexing
ss_values = social_security.values if hasattr(social_security, 'values') else social_security
taxable_ss_values = taxable_social_security.values if hasattr(taxable_social_security, 'values') else taxable_social_security

# Find problem indices
problem_mask = (ss_values > 0) & (np.abs(taxable_ss_values - ss_values) > 0.01)
problem_indices = np.where(problem_mask)[0]

fully_taxed_mask = (ss_values > 0) & (np.abs(taxable_ss_values - ss_values) <= 0.01)
fully_taxed_indices = np.where(fully_taxed_mask)[0]

print(f"Problem households: {len(problem_indices):,}")
print(f"Fully taxed households (for comparison): {len(fully_taxed_indices):,}")

Problem households: 13
Fully taxed households (for comparison): 6,930


### 6.1 Income Characteristics

In [7]:
# Calculate various income measures at household level
print("Calculating income characteristics...")

employment_income_hh = sim.calculate("employment_income", year, map_to="household")
agi_hh = sim.calculate("adjusted_gross_income", year, map_to="household")
taxable_income_hh = sim.calculate("taxable_income", year, map_to="household")

print("Done!")

Calculating income characteristics...
Done!


In [8]:
# Compare income characteristics between problem and fully taxed households
def print_comparison(var_name, var_array, problem_idx, fully_taxed_idx):
    """Print comparison statistics for a variable."""
    # Convert to numpy array for position-based indexing
    var_values = var_array.values if hasattr(var_array, 'values') else var_array
    
    problem_values = var_values[problem_idx]
    fully_taxed_values = var_values[fully_taxed_idx]
    
    print(f"\n{var_name}:")
    print(f"  Problem households ({len(problem_idx):,}):")
    print(f"    Mean:   ${problem_values.mean():>15,.2f}")
    print(f"    Median: ${np.median(problem_values):>15,.2f}")
    print(f"    Min:    ${problem_values.min():>15,.2f}")
    print(f"    Max:    ${problem_values.max():>15,.2f}")
    print(f"  Fully taxed households ({len(fully_taxed_idx):,}):")
    print(f"    Mean:   ${fully_taxed_values.mean():>15,.2f}")
    print(f"    Median: ${np.median(fully_taxed_values):>15,.2f}")

print("="*80)
print("INCOME CHARACTERISTICS COMPARISON")
print("="*80)

print_comparison("Social Security Benefits", social_security, problem_indices, fully_taxed_indices)
print_comparison("Taxable Social Security", taxable_social_security, problem_indices, fully_taxed_indices)
print_comparison("Employment Income (HH)", employment_income_hh, problem_indices, fully_taxed_indices)
print_comparison("Adjusted Gross Income (HH)", agi_hh, problem_indices, fully_taxed_indices)
print_comparison("Taxable Income (HH)", taxable_income_hh, problem_indices, fully_taxed_indices)

INCOME CHARACTERISTICS COMPARISON

Social Security Benefits:
  Problem households (13):
    Mean:   $      19,742.82
    Median: $      13,730.15
    Min:    $       4,748.52
    Max:    $      67,797.36
  Fully taxed households (6,930):
    Mean:   $      29,570.82
    Median: $      26,294.29

Taxable Social Security:
  Problem households (13):
    Mean:   $       7,012.94
    Median: $           0.00
    Min:    $           0.00
    Max:    $      28,829.09
  Fully taxed households (6,930):
    Mean:   $      29,570.82
    Median: $      26,294.29

Employment Income (HH):
  Problem households (13):
    Mean:   $      75,037.69
    Median: $      82,310.63
    Min:    $           0.00
    Max:    $     165,186.17
  Fully taxed households (6,930):
    Mean:   $      38,494.91
    Median: $           0.00

Adjusted Gross Income (HH):
  Problem households (13):
    Mean:   $      87,502.11
    Median: $      76,979.64
    Min:    $       8,045.53
    Max:    $     183,598.02
  Fully tax

### 6.2 Demographics and Filing Status

In [9]:
# Calculate demographic characteristics at tax_unit level, then map to household
# Get filing status from tax units
filing_status_tu = sim.calculate("filing_status", year, decode_enums=False)
tax_unit_ids = sim.calculate("tax_unit_id", year)
household_ids = sim.calculate("household_id", year)

# Create a mapping from household to tax unit filing status
# For simplicity, we'll take the first tax unit's filing status for each household
import pandas as pd
mapping_df = pd.DataFrame({
    'household_id': household_ids,
    'tax_unit_id': tax_unit_ids,
    'filing_status': filing_status_tu
})

# Get unique household-tax_unit combinations (take first tax unit per household)
household_filing = mapping_df.groupby('household_id').first()['filing_status'].values

# Convert to numpy array
filing_status_values = household_filing

filing_status_map = {
    0: "SINGLE",
    1: "JOINT",
    2: "SEPARATE",
    3: "HEAD_OF_HOUSEHOLD",
    4: "WIDOW"
}

print("\nFiling Status Distribution:")
print("="*80)
print("\nProblem households:")
for status_code, status_name in filing_status_map.items():
    count = (filing_status_values[problem_indices] == status_code).sum()
    pct = count / len(problem_indices) * 100 if len(problem_indices) > 0 else 0
    print(f"  {status_name:20s}: {count:5.0f} ({pct:5.1f}%)")

print("\nFully taxed households:")
for status_code, status_name in filing_status_map.items():
    count = (filing_status_values[fully_taxed_indices] == status_code).sum()
    pct = count / len(fully_taxed_indices) * 100 if len(fully_taxed_indices) > 0 else 0
    print(f"  {status_name:20s}: {count:>10,.0f} ({pct:5.1f}%)")


Filing Status Distribution:

Problem households:
  SINGLE              :    10 ( 76.9%)
  JOINT               :     2 ( 15.4%)
  SEPARATE            :     0 (  0.0%)
  HEAD_OF_HOUSEHOLD   :     1 (  7.7%)
  WIDOW               :     0 (  0.0%)

Fully taxed households:
  SINGLE              :      3,762 ( 54.3%)
  JOINT               :      2,449 ( 35.3%)
  SEPARATE            :        126 (  1.8%)
  HEAD_OF_HOUSEHOLD   :        576 (  8.3%)
  WIDOW               :         17 (  0.2%)


### 6.3 Edge Cases and Taxability Ratios

In [10]:
# Check for edge cases in problem households
print("Edge Case Analysis for Problem Households:")
print("="*80)

# Convert to numpy arrays for position-based indexing
agi_values = agi_hh.values if hasattr(agi_hh, 'values') else agi_hh
taxable_income_values = taxable_income_hh.values if hasattr(taxable_income_hh, 'values') else taxable_income_hh

# Negative AGI
negative_agi = (agi_values[problem_indices] < 0).sum()
print(f"Households with negative AGI: {negative_agi} ({negative_agi/len(problem_indices)*100:.1f}%)")

# Zero or negative taxable income
zero_taxable = (taxable_income_values[problem_indices] <= 0).sum()
print(f"Households with zero/negative taxable income: {zero_taxable} ({zero_taxable/len(problem_indices)*100:.1f}%)")

# Very low income
low_agi = (agi_values[problem_indices] < 1000).sum()
print(f"Households with AGI < $1,000: {low_agi} ({low_agi/len(problem_indices)*100:.1f}%)")

# Check what percentage of SS is taxable for problem households
problem_ss = ss_values[problem_indices]
problem_taxable_ss = taxable_ss_values[problem_indices]
problem_ratio = np.divide(
    problem_taxable_ss, 
    problem_ss,
    out=np.zeros_like(problem_taxable_ss, dtype=float),
    where=problem_ss > 0
)

print(f"\nTaxability ratio distribution:")
print(f"  0% taxable (ratio = 0):        {(problem_ratio == 0).sum()}  ({(problem_ratio == 0).sum()/len(problem_ratio)*100:.1f}%)")
print(f"  1-50% taxable:                  {((problem_ratio > 0) & (problem_ratio <= 0.5)).sum()}  ({((problem_ratio > 0) & (problem_ratio <= 0.5)).sum()/len(problem_ratio)*100:.1f}%)")
print(f"  50-85% taxable:                 {((problem_ratio > 0.5) & (problem_ratio <= 0.85)).sum()}  ({((problem_ratio > 0.5) & (problem_ratio <= 0.85)).sum()/len(problem_ratio)*100:.1f}%)")
print(f"  85-99% taxable:                 {((problem_ratio > 0.85) & (problem_ratio < 0.99)).sum()}  ({((problem_ratio > 0.85) & (problem_ratio < 0.99)).sum()/len(problem_ratio)*100:.1f}%)")
print(f"  99-100% taxable (almost):       {((problem_ratio >= 0.99) & (problem_ratio <= 1.0)).sum()}  ({((problem_ratio >= 0.99) & (problem_ratio <= 1.0)).sum()/len(problem_ratio)*100:.1f}%)")
print(f"  >100% taxable (weird!):         {(problem_ratio > 1.0).sum()}  ({(problem_ratio > 1.0).sum()/len(problem_ratio)*100:.1f}%)")

Edge Case Analysis for Problem Households:
Households with negative AGI: 0 (0.0%)
Households with zero/negative taxable income: 4 (30.8%)
Households with AGI < $1,000: 0 (0.0%)

Taxability ratio distribution:
  0% taxable (ratio = 0):        9  (69.2%)
  1-50% taxable:                  0  (0.0%)
  50-85% taxable:                 2  (15.4%)
  85-99% taxable:                 2  (15.4%)
  99-100% taxable (almost):       0  (0.0%)
  >100% taxable (weird!):         0  (0.0%)


### 6.4 Detailed Look at Individual Problem Cases

In [11]:
# Create detailed table for problem households
num_to_show = min(30, len(problem_indices))

# Get values arrays for indexing
employment_values = employment_income_hh.values if hasattr(employment_income_hh, 'values') else employment_income_hh

problem_data = []
for idx in problem_indices[:num_to_show]:
    problem_data.append({
        'HH_Index': int(idx),
        'SS_Benefits': f"${ss_values[idx]:,.0f}",
        'Taxable_SS': f"${taxable_ss_values[idx]:,.0f}",
        'Ratio': f"{taxable_ss_values[idx] / ss_values[idx] if ss_values[idx] > 0 else 0:.3f}",
        'Employment': f"${employment_values[idx]:,.0f}",
        'AGI': f"${agi_values[idx]:,.0f}",
        'Taxable_Inc': f"${taxable_income_values[idx]:,.0f}",
        'Filing': filing_status_map.get(int(filing_status_values[idx]), 'UNK')
    })

problem_df = pd.DataFrame(problem_data)
print(f"\nFirst {num_to_show} Problem Households:")
print("="*80)
print(problem_df.to_string(index=False))


First 13 Problem Households:
 HH_Index SS_Benefits Taxable_SS Ratio Employment      AGI Taxable_Inc            Filing
     2631     $33,578    $28,829 0.859   $149,177 $183,598    $119,829             JOINT
     5252     $13,730         $0 0.000   $165,186 $165,896    $121,514            SINGLE
     6304      $7,021     $4,994 0.711         $0  $15,771          $0            SINGLE
     7451      $4,753         $0 0.000    $82,311  $76,980     $42,970            SINGLE
     7764     $40,937    $28,516 0.697         $0  $28,516          $0            SINGLE
     8701     $14,862         $0 0.000   $148,080 $144,471    $119,971            SINGLE
     8773      $4,749         $0 0.000         $0  $60,370     $28,696            SINGLE
    14244     $12,421         $0 0.000    $21,745  $14,308          $0 HEAD_OF_HOUSEHOLD
    14534     $33,578    $28,829 0.859    $94,587 $136,969     $70,172            SINGLE
    16163     $13,730         $0 0.000   $130,795 $123,107     $87,541          

### 6.5 Other Income Sources and Deductions

In [12]:
# Check for other income sources that might affect SS taxation
print("Checking other income sources...")
print("="*80)

# Calculate various income types at household level
interest_income_hh = sim.calculate("taxable_interest_income", year, map_to="household")
dividend_income_hh = sim.calculate("qualified_dividend_income", year, map_to="household")
capital_gains_hh = sim.calculate("long_term_capital_gains", year, map_to="household")
pension_income_hh = sim.calculate("pension_income", year, map_to="household")

# Convert to numpy arrays
interest_values = interest_income_hh.values if hasattr(interest_income_hh, 'values') else interest_income_hh
dividend_values = dividend_income_hh.values if hasattr(dividend_income_hh, 'values') else dividend_income_hh
capital_gains_values = capital_gains_hh.values if hasattr(capital_gains_hh, 'values') else capital_gains_hh
pension_values = pension_income_hh.values if hasattr(pension_income_hh, 'values') else pension_income_hh

print("\nOther Income Sources - Problem Households:")
print(f"  Interest income > $0:          {(interest_values[problem_indices] > 0).sum()} households")
print(f"  Dividend income > $0:          {(dividend_values[problem_indices] > 0).sum()} households")
print(f"  Capital gains > $0:            {(capital_gains_values[problem_indices] > 0).sum()} households")
print(f"  Pension income > $0:           {(pension_values[problem_indices] > 0).sum()} households")

# Check deductions
standard_deduction_hh = sim.calculate("standard_deduction", year, map_to="household")

# Convert to numpy arrays
std_ded_values = standard_deduction_hh.values if hasattr(standard_deduction_hh, 'values') else standard_deduction_hh

print("\nDeductions - Problem Households:")
print(f"  Mean standard deduction:       ${std_ded_values[problem_indices].mean():,.2f}")

Checking other income sources...

Other Income Sources - Problem Households:
  Interest income > $0:          9 households
  Dividend income > $0:          2 households
  Capital gains > $0:            1 households
  Pension income > $0:           3 households

Deductions - Problem Households:
  Mean standard deduction:       $63,557.69


### 6.6 Key Findings Summary

In [13]:
print("\n" + "="*80)
print("KEY FINDINGS FROM INVESTIGATION")
print("="*80)

print(f"\n{len(problem_indices)} households have taxable_ss != ss, representing:")
print(f"  - ${ss_values[problem_indices].sum():,.0f} in SS benefits ({ss_values[problem_indices].sum()/total_ss*100:.1f}% of total)")
print(f"  - Only ${taxable_ss_values[problem_indices].sum():,.0f} taxable ({taxable_ss_values[problem_indices].sum()/ss_values[problem_indices].sum()*100:.1f}% of their benefits)")
print(f"  - Missing ${(ss_values[problem_indices] - taxable_ss_values[problem_indices]).sum():,.0f}")

print(f"\nPrimary characteristics of problem households:")
print(f"  - {zero_taxable} households ({zero_taxable/len(problem_indices)*100:.0f}%) have zero/negative taxable income")

zero_ratio_count = (problem_ratio == 0).sum()
print(f"  - {zero_ratio_count} households ({zero_ratio_count/len(problem_indices)*100:.0f}%) have 0% SS taxation")

weird_ratio_count = (problem_ratio > 1.0).sum()
if weird_ratio_count > 0:
    print(f"  - {weird_ratio_count} households have >100% taxation ratio (data quality issue?)")

print(f"\nLikely reasons for non-taxation:")
print(f"  1. Zero taxable income after deductions (can't tax SS if total taxable income is zero)")
print(f"  2. Negative AGI from losses or deductions")
print(f"  3. Possible data quality issues in the microsimulation weights/values")

print(f"\nConclusion:")
print(f"  Option 8 achieves {percentage_taxable:.2f}% taxation at the national level.")
print(f"  The {100-percentage_taxable:.2f}% gap is primarily due to households with no taxable income,")
print(f"  where SS benefits cannot be taxed because total taxable income is zero/negative.")
print("="*80)


KEY FINDINGS FROM INVESTIGATION

13 households have taxable_ss != ss, representing:
  - $256,657 in SS benefits (0.0% of total)
  - Only $91,168 taxable (35.5% of their benefits)
  - Missing $165,488

Primary characteristics of problem households:
  - 4 households (31%) have zero/negative taxable income
  - 9 households (69%) have 0% SS taxation

Likely reasons for non-taxation:
  1. Zero taxable income after deductions (can't tax SS if total taxable income is zero)
  2. Negative AGI from losses or deductions
  3. Possible data quality issues in the microsimulation weights/values

Conclusion:
  Option 8 achieves 99.89% taxation at the national level.
  The 0.11% gap is primarily due to households with no taxable income,
  where SS benefits cannot be taxed because total taxable income is zero/negative.


## 7. Final Summary

In [14]:
print("\n" + "="*80)
if abs(difference) < 1000:
    print("[PASS] Option 8 correctly taxes 100% of Social Security benefits nationwide")
    print("       (within rounding tolerance)")
else:
    print("[FAIL] Option 8 does NOT tax 100% of Social Security benefits")
    print(f"       Difference of ${difference:,.0f} ({100-percentage_taxable:.2f}% untaxed)")
    print(f"       This affects {len(problem_indices):,} households")
    print(f"\n       However, this is expected behavior:")
    print(f"       - {zero_taxable} of these households have zero/negative taxable income")
    print(f"       - SS benefits cannot be taxed if total taxable income is zero")
    print(f"       - Option 8 reform is configured correctly for 100% taxation")
print("="*80)


[FAIL] Option 8 does NOT tax 100% of Social Security benefits
       Difference of $1,612,271,577 (0.11% untaxed)
       This affects 13 households

       However, this is expected behavior:
       - 4 of these households have zero/negative taxable income
       - SS benefits cannot be taxed if total taxable income is zero
       - Option 8 reform is configured correctly for 100% taxation
